In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
# !pip3 install albert-tensorflow

In [3]:
from albert import modeling
from albert import optimization
from albert import tokenization
import tensorflow as tf
import numpy as np

In [4]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='albert-base-2020-04-10/sp10m.cased.v10.vocab', do_lower_case=False,
      spm_model_file='albert-base-2020-04-10/sp10m.cased.v10.model')


INFO:tensorflow:loading sentence piece model


In [5]:
albert_config = modeling.AlbertConfig.from_json_file('albert-base-2020-04-10/config.json')
albert_config

In [6]:
from rules import normalized_chars
import random
from unidecode import unidecode
import re

laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [7]:
import json
import random

emotion_label = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise']

with open('../sentiment/emotion-twitter-lexicon.json') as fopen:
    emotion = json.load(fopen)
    
emotion.keys()

dict_keys(['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'])

In [8]:
texts, labels = [], []

for k, v in emotion.items():
    if len(v) > 30000:
        emotion[k] = random.sample(v, 30000)
    print(k, len(emotion[k]))
    texts.extend(emotion[k])
    labels.extend([emotion_label.index(k)] * len(emotion[k]))

anger 30000
fear 20316
happy 30000
love 20783
sadness 26468
surprise 13107


In [9]:
from tqdm import tqdm

for i in tqdm(range(len(texts))):
    texts[i] = cleaning(texts[i])

100%|██████████| 140674/140674 [00:11<00:00, 11765.54it/s]


In [10]:
actual_t, actual_l = [], []

for i in tqdm(range(len(texts))):
    if len(texts[i]) > 2:
        actual_t.append(texts[i])
        actual_l.append(labels[i])

100%|██████████| 140674/140674 [00:00<00:00, 1369422.83it/s]


In [11]:
from tqdm import tqdm

input_ids, input_masks = [], []

for text in tqdm(actual_t):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    input_masks.append(input_mask)

100%|██████████| 140674/140674 [00:19<00:00, 7400.09it/s]


In [12]:
epoch = 3
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [13]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.MASK = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.AlbertModel(
            config=albert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.MASK,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            albert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
BERT_INIT_CHKPNT = 'albert-base-2020-04-10/model.ckpt-400000'

In [15]:
dimension_output = 6
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:++++++ warmup starts at step 0, for 703 steps ++++++
INFO:tensorflow:using adamw

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from albert-base-2020-04-10/model.ckpt-400000


In [16]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y, train_mask, test_mask = train_test_split(
    input_ids, actual_l, input_masks, test_size = 0.2
)

In [17]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [18]:
from tqdm import tqdm
import time

for EPOCH in range(epoch):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = train_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = test_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 0/1876 [00:00<?, ?it/s]

epoch: 0, training loss: 0.126535, training acc: 0.961132, valid loss: 0.017495, valid acc: 0.995700



train minibatch loop:   0%|          | 0/1876 [00:00<?, ?it/s]

epoch: 1, training loss: 0.010664, training acc: 0.997610, valid loss: 0.016509, valid acc: 0.996553



test minibatch loop: 100%|██████████| 469/469 [00:51<00:00,  9.20it/s, accuracy=1, cost=0.00112]   

epoch: 2, training loss: 0.003345, training acc: 0.999289, valid loss: 0.012979, valid acc: 0.997584



In [19]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'albert-base-emotion/model.ckpt')

'albert-base-emotion/model.ckpt'

In [20]:
dimension_output = 6
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'albert-base-emotion/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:++++++ warmup starts at step 0, for 703 steps ++++++
INFO:tensorflow:using adamw
INFO:tensorflow:Restoring parameters from albert-base-emotion/model.ckpt


In [21]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/embedding_hidden_mapping_in/kernel',
 'bert/encoder/embedding_hidden_mapping_in/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/bias',
 'bert/encoder/transformer/group_0/inner_group_0/Lay

In [22]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [23]:
freeze_graph('albert-base-emotion', strings)

INFO:tensorflow:Restoring parameters from albert-base-emotion/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 29 variables.
INFO:tensorflow:Converted 29 variables to const ops.
3242 ops in the final graph.


In [24]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [25]:
g = load_graph('albert-base-emotion/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
mask = g.get_tensor_by_name('import/Placeholder_1:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)
result = test_sess.run(tf.nn.softmax(logits), feed_dict = {x: [input_id], mask: [input_mask]})
result

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([[7.0160480e-05, 2.6400085e-05, 1.9688978e-05, 1.4426747e-05,
        2.0472557e-05, 9.9984884e-01]], dtype=float32)

In [26]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'albert-base-emotion/frozen_model.pb'
outPutname = "v34/emotion/albert-base-emotion.pb"

s3 = boto3.client('s3')
s3.upload_file(Key,bucketName,outPutname)

In [27]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_mask = test_mask[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            model.MASK: batch_mask
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 469/469 [00:50<00:00,  9.29it/s]


In [28]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
        digits = 5
    )
)

              precision    recall  f1-score   support

       anger    0.99785   0.99472   0.99628      6062
        fear    0.99582   0.99926   0.99754      4056
       happy    0.99866   0.99866   0.99866      5988
        love    0.99712   0.99760   0.99736      4162
     sadness    0.99813   0.99813   0.99813      5334
    surprise    0.99685   0.99803   0.99744      2533

    accuracy                        0.99758     28135
   macro avg    0.99740   0.99773   0.99757     28135
weighted avg    0.99758   0.99758   0.99758     28135

